In [ ]:
import pandas as pd
from pathlib import Path
from PIL import Image, ImageChops, ImageDraw
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

root_dir = Path("data/20241121_reference")
# csv_path = Path("data/merged_reference_data.csv")
# alignment_data = pd.read_csv(csv_path)
  
# def apply_diff_as_mask(before_diff_image, diff_image,threshhold):
    
#     diff_image_array = np.array(diff_image)
    
#     mask = diff_image_array > threshhold

#     before_diff_image_array = np.array(before_diff_image)

#     masked_image = np.copy(before_diff_image_array)
#     masked_image[~mask] = 0 

#     return masked_image, mask

def show_images(before_diff_image, diff_image, masked_image):

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    titles = ['Original Image', 'Difference Image', 'Masked Image']
    images = [before_diff_image, diff_image, masked_image]

    for ax, image, title in zip(axes, images, titles):
        ax.imshow(image, cmap='gray')
        ax.set_title(title)
        ax.axis('off')

    plt.tight_layout()
    plt.show()
#########################################################################################################
#alignment_data["File_Area"] = alignment_data["File"].astype(str) + "_" + alignment_data["Area"].astype(str)

for folder in root_dir.iterdir():
    if folder.is_dir(): 
                       
        images = list(folder.glob("*.tif"))
        if len(images) != 2:
            print(f"Skipping {folder}, as it doesn't contain exactly 2 images.")
            continue

        base_name = folder.name
        base_image_path = next((img for img in images if base_name in img.stem), None)
        if not base_image_path:
            print(f"No base image matching folder name found in {folder}. Skipping.")
            continue
        
        other_image_path = next((img for img in images if img != base_image_path), None)

        base_image_name = base_image_path.stem
        other_image_name = other_image_path.stem

        # try:
        #     base_row = alignment_data[alignment_data["File_Area"] == base_image_name].iloc[0]
        #     other_row = alignment_data[alignment_data["File_Area"] == other_image_name].iloc[0]
        # except IndexError:
        #     print(f"No alignment data found for {base_image_name} or {other_image_name}. Skipping this folder.")
        #     continue

        # base_area = base_row["Area"]
        # is_vertical = base_area.startswith("C")

        base_distance = base_row["Data"]
        other_distance = other_row["Data"]         
        shift_distance = int(np.round(other_distance - base_distance))#distance from right side or up side
        

        base_image = Image.open(base_image_path).convert("RGB")
        other_image = Image.open(other_image_path).convert("RGB")
            
##########################################################################################     
# 现在我需要写一个python函数用来合成两张图片
 
        gap_mask = np.zeros((other_image.size[1], other_image.size[0]), dtype=bool) 
        
        if is_vertical:
            
            other_image_shifted = ImageChops.offset(other_image, 0, -shift_distance)  
            
            if shift_distance > 0:
                gap_mask[-shift_distance:, :] = True
            elif shift_distance < 0:
                gap_mask[:-shift_distance, :] = True
        else:
            
            other_image_shifted = ImageChops.offset(other_image, shift_distance, 0)
            print(f"base_distance:{base_distance};other_distance:{other_distance}")
            display(other_image_shifted)
            if shift_distance > 0:
                gap_mask[:, :shift_distance] = True
            elif shift_distance < 0:
                gap_mask[:, shift_distance:] = True

        diff_image = ImageChops.difference(base_image, other_image_shifted)
        #diff_image = ImageChops.subtract(other_image_shifted,base_image,  scale=1, offset=0)
        
 ##########################################################################################       
        print(f"Anomaly Image ({base_image_name}):")
        save_image_with_overwrite(base_image,folder / "result/Anomaly Image.png")  #1             
        print(f"Reference Image ({other_image_name}) by {shift_distance}:")
        save_image_with_overwrite(other_image,folder / "result/Reference Image.png") #2           
        diff_pixels = np.array(diff_image)
        diff_pixels[gap_mask] = [0,0,0] 
        diff_image = Image.fromarray(diff_pixels, mode="RGB")
        print(f"Difference Image after masking:")
        save_image_with_overwrite(diff_image,folder / "result/aligned_diff_image.png") #3
        
 ############################################################################################# 
 #mask抽出      
        masked_image, mask = apply_diff_as_mask(base_image, diff_image,threshhold=20)
    
        mask_image = np.uint8(mask) * 255 
        mask_pil_image = Image.fromarray(mask_image)
        save_image_with_overwrite(mask_pil_image, folder/"result/mask_image.png") 
   
        masked_pil_image = Image.fromarray(masked_image)
        save_image_with_overwrite(masked_pil_image, folder/"result/masked_diff_image.png")        
        #show_images(base_image, diff_image, masked_image)   
